3.6.2 신경망의 추론 처리
    - MNIST 데이터셋의 경우, 입력층 뉴런 수 784(28*28) + 출력층 뉴런 수 10(0~9)
    - 은닉 층의 뉴런 수는 우선 임의로 지정(50, 100)

In [13]:
import numpy as np

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def softmax(a):
    c=np.max(a)
    exp_a=np.exp(a-c) # 오버플로 방지
    sum_exp_a=np.sum(exp_a)
    y=exp_a/sum_exp_a    

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
# normalize 옵션으로 픽셀의 값을 0.0~ 1.0 범위로 변환 == 전처리 작업으로 정규화 수행
    return x_test, t_test

def init_network(): # pickle 파일인 sample_weight.pkl에 저장된 학습된 가중치 매개변수를 읽어옴
    with open("sample_weight.pkl", 'rb') as f:
        network=pickle.load(f)  # 가중치와 편향 매개변수가 딕셔너리 변수로 저장되어 있음

    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1=np.dot(x, W1) + b1
    z1=sigmoid(a1)
    a2=np.dot(z1,W2) + b2
    z2=sigmoid(a2)
    a3=np.dot(z2,W3) + b3
    y=softmax(a3)

    return y

* 전처리: 신경망의 입력 데이터에 특정 변환을 가하는 것
    - 정규화: 데이터를 특정 범위로 변환하는 처리
    - 백색화: 전체 데이터를 균일하게 분포하도록 처리

In [14]:
import sys, os
sys.path.append(os.pardir)
from mnist import load_mnist

import pickle

# 정확도 평가
x, t = get_data()               # 데이터셋 획득
network = init_network()        # 네트워크 생성

accuracy_cnt=0
for i in range(len(x)):
    y=predict(network, x[i])    # 이미지 데이터를 하나씩 predict() 함수로 분류-> 각 레이블의 확률을 넘파이 배열로 반환
    p=np.argmax(y)              # 확률이 가장 높은 원소의 인덱스 추츨 == 예측 결과
    if p==t[i]:
        accuracy_cnt+=1         # 신경망 예측과 정답 레이블 비교 후 정확도 업데이트

print("Accuracy:" + str(float(accuracy_cnt)/len(x)))

Accuracy:0.098


3.6.3 배치 처리

In [16]:
x, _ = get_data()
network=init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)
# 다차원 배열의 대응하는 차원의 원소 수 일치 확인
# 최종 결과로 원소가 10개인 1차원 배열 출력

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


- batch: 하나로 묶은 입력 데이터
- 배치 처리의 이점
    1. 이미지 1장당 처리 시간을 대폭 축소
         - 수치 계산 라이브러리 대부분이 큰 배열을 효율적으로 처리할 수 있도록 최적화되어 있음
         - 데이터 전송의 병목 현상을 줄일 수 있음
         = 느린 입출력 장치를 통해 데이터를 읽는 횟수에 비해 CPU 및 GPU로 순수 계산을 수행하는 비율 향상